# Laboratorio 2 - Parte 1

In [1]:
import pandas as pd
import sys
import time

import matplotlib.pyplot as plt
import seaborn as sns

#matplotlib inline
import numpy as np
import pandas as pd
from IPython.display import HTML

sys.path.append("code/.")

#import mglearn
from IPython.display import display
#from plotting_functions import *

# Classifiers and regressors
from sklearn.dummy import DummyClassifier, DummyRegressor

# Preprocessing and pipeline
from sklearn.impute import SimpleImputer

# train test split and cross validation
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
#from utils import *

pd.set_option("display.max_colwidth", 200)

In [2]:
# Importemos los datos:
df = pd.read_csv('census.csv')

df.head()


,age,workclass,education,race,sex,hours_per_week,USA_born,label
0,39.0,State-gov,Bachelors,White,Male,40.0,1.0,<=50K
1,50.0,Self-emp-not-inc,Bachelors,White,Male,13.0,1.0,<=50K
2,38.0,Private,High-school,White,Male,40.0,1.0,<=50K
3,53.0,Private,Some-high-school,Black,Male,40.0,1.0,<=50K
4,28.0,Private,Bachelors,Black,Female,40.0,0.0,<=50K


In [3]:
# Veamos el dataset:
df

,age,workclass,education,race,sex,hours_per_week,USA_born,label
0,39.0,State-gov,Bachelors,White,Male,40.0,1.0,<=50K
1,50.0,Self-emp-not-inc,Bachelors,White,Male,13.0,1.0,<=50K
2,38.0,Private,High-school,White,Male,40.0,1.0,<=50K
3,53.0,Private,Some-high-school,Black,Male,40.0,1.0,<=50K
4,28.0,Private,Bachelors,Black,Female,40.0,0.0,<=50K
...,...,...,...,...,...,...,...,...
41711,33.0,Private,Bachelors,White,Male,40.0,1.0,<=50K
41712,39.0,Private,Bachelors,White,Female,36.0,1.0,<=50K
41713,38.0,Private,Bachelors,White,Male,50.0,1.0,<=50K
41714,44.0,Private,Bachelors,Asian-Pac-Islander,Male,40.0,1.0,<=50K


In [4]:
# Descripción de las columnas:

df.describe(include='all')

,age,workclass,education,race,sex,hours_per_week,USA_born,label
count,41617.000000,41705,41702,41700,41701,41631.000000,41701.000000,41716
unique,NaN,7,10,5,2,NaN,NaN,2
top,NaN,Private,High-school,White,Male,NaN,NaN,<=50K
freq,NaN,30962,14969,35670,28191,NaN,NaN,31813
mean,38.476608,NaN,NaN,NaN,NaN,40.743220,0.895062,NaN
std,13.365972,NaN,NaN,NaN,NaN,12.000085,0.306477,NaN
min,17.000000,NaN,NaN,NaN,NaN,1.000000,0.000000,NaN
25%,28.000000,NaN,NaN,NaN,NaN,40.000000,1.000000,NaN
50%,37.000000,NaN,NaN,NaN,NaN,40.000000,1.000000,NaN
75%,47.000000,NaN,NaN,NaN,NaN,45.000000,1.000000,NaN


In [5]:
# Verifiquemos si hay datos faltantes:

df.isnull().sum()


age               99
workclass         11
education         14
race              16
sex               15
hours_per_week    85
USA_born          15
label              0
dtype: int64

In [6]:
# Descartemos las filas que tengan 3 o más datos faltantes:
df = df[df.isnull().sum(axis=1) < 3]
df

,age,workclass,education,race,sex,hours_per_week,USA_born,label
0,39.0,State-gov,Bachelors,White,Male,40.0,1.0,<=50K
1,50.0,Self-emp-not-inc,Bachelors,White,Male,13.0,1.0,<=50K
2,38.0,Private,High-school,White,Male,40.0,1.0,<=50K
3,53.0,Private,Some-high-school,Black,Male,40.0,1.0,<=50K
4,28.0,Private,Bachelors,Black,Female,40.0,0.0,<=50K
...,...,...,...,...,...,...,...,...
41711,33.0,Private,Bachelors,White,Male,40.0,1.0,<=50K
41712,39.0,Private,Bachelors,White,Female,36.0,1.0,<=50K
41713,38.0,Private,Bachelors,White,Male,50.0,1.0,<=50K
41714,44.0,Private,Bachelors,Asian-Pac-Islander,Male,40.0,1.0,<=50K


In [7]:
# Contemos de nuevo los datos faltantes:
df.isnull().sum()

age               85
workclass          0
education          0
race               0
sex                0
hours_per_week    68
USA_born           0
label              0
dtype: int64

In [8]:
# Imputemos los datos faltantes de edad y horas trabajadas por semana con la mediana de cada una de esas columnas:
df['age'] = df['age'].fillna(df['age'].median())
df['hours_per_week'] = df['hours_per_week'].fillna(df['hours_per_week'].median())
df

C:\Users\danna\AppData\Local\Temp\ipykernel_37632\123229190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = df['age'].fillna(df['age'].median())
C:\Users\danna\AppData\Local\Temp\ipykernel_37632\123229190.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hours_per_week'] = df['hours_per_week'].fillna(df['hours_per_week'].median())


,age,workclass,education,race,sex,hours_per_week,USA_born,label
0,39.0,State-gov,Bachelors,White,Male,40.0,1.0,<=50K
1,50.0,Self-emp-not-inc,Bachelors,White,Male,13.0,1.0,<=50K
2,38.0,Private,High-school,White,Male,40.0,1.0,<=50K
3,53.0,Private,Some-high-school,Black,Male,40.0,1.0,<=50K
4,28.0,Private,Bachelors,Black,Female,40.0,0.0,<=50K
...,...,...,...,...,...,...,...,...
41711,33.0,Private,Bachelors,White,Male,40.0,1.0,<=50K
41712,39.0,Private,Bachelors,White,Female,36.0,1.0,<=50K
41713,38.0,Private,Bachelors,White,Male,50.0,1.0,<=50K
41714,44.0,Private,Bachelors,Asian-Pac-Islander,Male,40.0,1.0,<=50K


In [9]:
# Contemos de nuevo los datos faltantes:
df.isnull().sum()

age               0
workclass         0
education         0
race              0
sex               0
hours_per_week    0
USA_born          0
label             0
dtype: int64

In [10]:
# Apliquemos one-hot encoding a la columna "workclass":

df = pd.get_dummies(df, columns=['workclass'], prefix='workclass')
df.head()


,age,education,race,sex,hours_per_week,USA_born,label,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,39.0,Bachelors,White,Male,40.0,1.0,<=50K,False,False,False,False,False,True,False
1,50.0,Bachelors,White,Male,13.0,1.0,<=50K,False,False,False,False,True,False,False
2,38.0,High-school,White,Male,40.0,1.0,<=50K,False,False,True,False,False,False,False
3,53.0,Some-high-school,Black,Male,40.0,1.0,<=50K,False,False,True,False,False,False,False
4,28.0,Bachelors,Black,Female,40.0,0.0,<=50K,False,False,True,False,False,False,False


In [11]:
# Hallemos los valores que toma la columna "education":
df["education"].unique()



array(['Bachelors', 'High-school', 'Some-high-school', 'Masters',
       'Some-college', 'Middle-school', 'Doctorate', 'Some-middle-school',
       'Preschool', 'Elementary-school'], dtype=object)

In [12]:
# Apliquemos ordinal encoding a la columna "education":

from sklearn.preprocessing import OrdinalEncoder

orden_education = [
    'Preschool',
    'Elementary-school',
    'Middle-school',
    'Some-middle-school',
    'High-school',
    'Some-high-school',
    'Some-college',
    'Bachelors',
    'Masters',
    'Doctorate'
]

encoder_ordinal = OrdinalEncoder(categories=[orden_education])

# Imputamos valores faltantes si hay NaN
df['education'] = df['education'].fillna('Preschool')

# Aplicamos la codificación
df['education'] = encoder_ordinal.fit_transform(df[['education']])

# Verificamos los primeros resultados
df[['education']].head()

,education
0,7.0
1,7.0
2,4.0
3,5.0
4,7.0


In [13]:
# Verifiquemos que la columna "education" tenga los valores apropiados:
print(df['education'].unique())


[7. 4. 5. 8. 6. 2. 9. 3. 0. 1.]


In [14]:
# Apliquemos one-hot encoding a la columna "race":
df = pd.get_dummies(df, columns=['race'])
df

,age,education,sex,hours_per_week,USA_born,label,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,39.0,7.0,Male,40.0,1.0,<=50K,False,False,False,False,False,True,False,False,False,False,False,True
1,50.0,7.0,Male,13.0,1.0,<=50K,False,False,False,False,True,False,False,False,False,False,False,True
2,38.0,4.0,Male,40.0,1.0,<=50K,False,False,True,False,False,False,False,False,False,False,False,True
3,53.0,5.0,Male,40.0,1.0,<=50K,False,False,True,False,False,False,False,False,False,True,False,False
4,28.0,7.0,Female,40.0,0.0,<=50K,False,False,True,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41711,33.0,7.0,Male,40.0,1.0,<=50K,False,False,True,False,False,False,False,False,False,False,False,True
41712,39.0,7.0,Female,36.0,1.0,<=50K,False,False,True,False,False,False,False,False,False,False,False,True
41713,38.0,7.0,Male,50.0,1.0,<=50K,False,False,True,False,False,False,False,False,False,False,False,True
41714,44.0,7.0,Male,40.0,1.0,<=50K,False,False,True,False,False,False,False,False,True,False,False,False


In [15]:
# Apliquemos binary encoding a la columna "sex":
df['sex'] = df['sex'].map({'Male': 1, 'Female': 0})
df

,age,education,sex,hours_per_week,USA_born,label,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,39.0,7.0,1,40.0,1.0,<=50K,False,False,False,False,False,True,False,False,False,False,False,True
1,50.0,7.0,1,13.0,1.0,<=50K,False,False,False,False,True,False,False,False,False,False,False,True
2,38.0,4.0,1,40.0,1.0,<=50K,False,False,True,False,False,False,False,False,False,False,False,True
3,53.0,5.0,1,40.0,1.0,<=50K,False,False,True,False,False,False,False,False,False,True,False,False
4,28.0,7.0,0,40.0,0.0,<=50K,False,False,True,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41711,33.0,7.0,1,40.0,1.0,<=50K,False,False,True,False,False,False,False,False,False,False,False,True
41712,39.0,7.0,0,36.0,1.0,<=50K,False,False,True,False,False,False,False,False,False,False,False,True
41713,38.0,7.0,1,50.0,1.0,<=50K,False,False,True,False,False,False,False,False,False,False,False,True
41714,44.0,7.0,1,40.0,1.0,<=50K,False,False,True,False,False,False,False,False,True,False,False,False


In [16]:
# Apliquemos binary encoding a la etiqueta:
df['label'] = df['label'].map({'>50K': 1, '<=50K': 0})
df

,age,education,sex,hours_per_week,USA_born,label,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,39.0,7.0,1,40.0,1.0,0,False,False,False,False,False,True,False,False,False,False,False,True
1,50.0,7.0,1,13.0,1.0,0,False,False,False,False,True,False,False,False,False,False,False,True
2,38.0,4.0,1,40.0,1.0,0,False,False,True,False,False,False,False,False,False,False,False,True
3,53.0,5.0,1,40.0,1.0,0,False,False,True,False,False,False,False,False,False,True,False,False
4,28.0,7.0,0,40.0,0.0,0,False,False,True,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41711,33.0,7.0,1,40.0,1.0,0,False,False,True,False,False,False,False,False,False,False,False,True
41712,39.0,7.0,0,36.0,1.0,0,False,False,True,False,False,False,False,False,False,False,False,True
41713,38.0,7.0,1,50.0,1.0,0,False,False,True,False,False,False,False,False,False,False,False,True
41714,44.0,7.0,1,40.0,1.0,0,False,False,True,False,False,False,False,False,True,False,False,False


In [17]:
# Veamos el dataset resultante:
df.head()

,age,education,sex,hours_per_week,USA_born,label,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,39.0,7.0,1,40.0,1.0,0,False,False,False,False,False,True,False,False,False,False,False,True
1,50.0,7.0,1,13.0,1.0,0,False,False,False,False,True,False,False,False,False,False,False,True
2,38.0,4.0,1,40.0,1.0,0,False,False,True,False,False,False,False,False,False,False,False,True
3,53.0,5.0,1,40.0,1.0,0,False,False,True,False,False,False,False,False,False,True,False,False
4,28.0,7.0,0,40.0,0.0,0,False,False,True,False,False,False,False,False,False,True,False,False


In [18]:
# Veamos la nueva descripción del dataset:
df.describe()

,age,education,sex,hours_per_week,USA_born,label
count,41694.000000,41694.000000,41694.000000,41694.000000,41694.000000,41694.000000
mean,38.471651,5.381782,0.676045,40.742529,0.895045,0.237372
std,13.351404,1.528790,0.467989,11.990589,0.306499,0.425477
min,17.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,28.000000,4.000000,0.000000,40.000000,1.000000,0.000000
50%,37.000000,6.000000,1.000000,40.000000,1.000000,0.000000
75%,47.000000,7.000000,1.000000,45.000000,1.000000,0.000000
max,90.000000,9.000000,1.000000,99.000000,1.000000,1.000000


In [19]:
# Verifiquemos el tipo de dato de cada columna:
df.dtypes

age                           float64
education                     float64
sex                             int64
hours_per_week                float64
USA_born                      float64
label                           int64
workclass_Federal-gov            bool
workclass_Local-gov              bool
workclass_Private                bool
workclass_Self-emp-inc           bool
workclass_Self-emp-not-inc       bool
workclass_State-gov              bool
workclass_Without-pay            bool
race_Amer-Indian-Eskimo          bool
race_Asian-Pac-Islander          bool
race_Black                       bool
race_Other                       bool
race_White                       bool
dtype: object

In [20]:
# Carguemos el dataset a un nuevo archivo:
df.to_csv('dataset_codificado.csv', index=False)
print("Dataset guardado como 'dataset_codificado.csv'")

Dataset guardado como 'dataset_codificado.csv'
